In [1]:
import sklearn.model_selection
import sklearn.model_selection
from load_data import *
from keras.models import Model
from keras.layers import Input, Dense
from sklearn.preprocessing import normalize
from load_data import load_all_feature_vectors

Using TensorFlow backend.


In [2]:
seiz_ad, non_seiz_ad = load_all_feature_vectors(all_bands=False)

Done with  1
Done with  2
Done with  3
Done with  4
Done with  5
Done with  7
Done with  9
Done with  11
Done with  14
Done with  16
Mistake Theta
Mistake Alpha
Mistake Theta
Mistake Alpha
Mistake Theta
Mistake Alpha
Done with  17
Done with  18
Done with  19


In [3]:
import pandas as pd
import os
from openpyxl import load_workbook

def save_ann_data_to(data, path):
    if os.path.isfile(path):
        book = load_workbook(path)
        writer = pd.ExcelWriter(path, engine='openpyxl')
        writer.book = book

        df = pd.DataFrame(data, columns=['Loss', 'binary_accuracy', 'precision', 'specificity'])
        df.to_excel(writer)

        writer.save()
        writer.close()

    else:
        df = pd.DataFrame(data,
                          columns=['Loss', 'binary_accuracy', 'precision', 'specificity'])
        df.to_excel(path)


def save_svm_data_to(data, path):
    if os.path.isfile(path):
        book = load_workbook(path)
        writer = pd.ExcelWriter(path, engine='openpyxl')
        writer.book = book

        df = pd.DataFrame(data, columns=['Mean Accuracy', 'Mean precision', 'Sensitivity', 'Specificity'])
        df.to_excel(writer)

        writer.save()
        writer.close()

    else:
        df = pd.DataFrame(data,
                          columns=['Mean Accuracy', 'Mean precision', 'Sensitivity', 'Specificity'])
        df.to_excel(path)

def save_rdf_data_to(data, path):
    if os.path.isfile(path):
        book = load_workbook(path)
        writer = pd.ExcelWriter(path, engine='openpyxl')
        writer.book = book

        df = pd.DataFrame(data, columns=['Mean Accuracy', 'Precision', 'Sensitivity', 'Specificity'])
        df.to_excel(writer)

        writer.save()
        writer.close()

    else:
        df = pd.DataFrame(data,
                          columns=['Mean Accuracy', 'Precision', 'Sensitivity', 'Specificity'])
        df.to_excel(path)


In [3]:
import keras.backend as K

def precision(y_true, y_pred):
    """Precision metric.
    Only computes a batch-wise average of precision.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    neg_y_true = 1 - y_true
    neg_y_pred = 1 - y_pred
    fp = K.sum(neg_y_true * y_pred)
    tn = K.sum(neg_y_true * neg_y_pred)
    tp = K.sum(y_true * y_pred)
    fn = K.sum(neg_y_pred * y_pred)
    precision = tp / (tp + fp + K.epsilon())
    return precision

def specificity(y_true, y_pred):
    """
    param:
    y_pred - Predicted labels
    y_true - True labels 
    Returns:
    Specificity score
    """
    neg_y_true = 1 - y_true
    neg_y_pred = 1 - y_pred
    fp = K.sum(neg_y_true * y_pred)
    tn = K.sum(neg_y_true * neg_y_pred)
    specificity = tn / (tn + fp + K.epsilon())
    return specificity

def sensitivity(y_true, y_pred):
    neg_y_true = 1 - y_true
    neg_y_pred = 1 - y_pred
    fp = K.sum(neg_y_true * y_pred)
    tn = K.sum(neg_y_true * neg_y_pred)
    tp = K.sum(y_true * y_pred)
    fn = K.sum(neg_y_pred * y_pred)
    sensitivity = tp / (tp + fn + K.epsilon())
    return sensitivity


In [6]:
def cutting(seiz, non_seiz, factor):
    index = np.random.randint(factor * len(seiz), len(non_seiz) - factor * len(seiz) - 1)
    stop = index + int(factor * len(seiz))
    non_seiz_vector = non_seiz[index:stop][:]

    #concattenate the newly created vecotrs for seiz and non_seiz
    seizure = []
    for i in range(len(seiz)):
        if len(seiz[i]) > 0:
            seizure.append(seiz[i])
    data = np.concatenate((np.asarray(seizure), np.asarray(non_seiz_vector)), axis=0)

    seiz_labels = np.ones(len(seizure))
    non_seiz_labels = np.zeros(len(non_seiz_vector))
    labels = np.concatenate((seiz_labels, non_seiz_labels))

    return normalize(data, 'l1'), labels

def norm(seiz, non_seiz):
    #create labels
    seizure = []
    for i in range(len(seiz)):
        if len(seiz[i]) > 0:
            seizure.append(seiz[i])

    seiz_labels = np.ones(len(seizure))
    non_seiz_labels = np.zeros(len(non_seiz))
    labels = np.concatenate((seiz_labels, non_seiz_labels))

    #concattenate the newly created vecotrs for seiz and non_seiz
    data = np.concatenate((np.asarray(seizure), np.asarray(non_seiz)), axis=0)

    return normalize(data, 'l1'), labels


In [4]:
metrics = ['binary_accuracy',precision, sensitivity, specificity]

In [ ]:
ann_all_data_alpha_delta = []
for i in range(100):
    ann_data, labels = norm(seiz_ad.copy(), non_seiz_ad.copy())
    ann_x_train, ann_x_test, ann_y_train, ann_y_test = sklearn.model_selection.train_test_split(ann_data, labels)

    inp = Input(shape=(6,))
    hidden = Dense(6, activation='sigmoid')(inp)
    outp = Dense(1)(hidden)
    model = Model(inputs=inp, outputs=outp)

    model.compile(optimizer='SGD',
                  loss='mean_squared_error',
                  metrics=metrics)  #Using MSE as lossfunction

    model.fit(x=ann_x_train, y=ann_y_train,
              batch_size=3,
              epochs=1000,
              shuffle=True,
              verbose=False)
    score = model.evaluate(x=ann_x_test, y=ann_y_test, verbose=False)
    ann_all_data_alpha_delta.append(score)


In [15]:
save_ann_data_to(ann_all_data_alpha_delta, 'ANN_all_data_alpha_delta.xlsx')

In [ ]:
ann_all_data_alpha_delta_3t1 = []
for i in range(100):
    ann_data, labels = cutting(seiz_ad.copy(), non_seiz_ad.copy(), factor=3)
    ann_x_train, ann_x_test, ann_y_train, ann_y_test = sklearn.model_selection.train_test_split(ann_data, labels)

    inp = Input(shape=(6,))
    hidden = Dense(6, activation='sigmoid')(inp)
    outp = Dense(1)(hidden)
    model = Model(inputs=inp, outputs=outp)

    model.compile(optimizer='SGD',
                  loss='mean_squared_error',
                  metrics=metrics)  #Using MSE as lossfunction

    model.fit(x=ann_x_train, y=ann_y_train,
              batch_size=3,
              epochs=1000,
              shuffle=True,
              verbose=False)
    score = model.evaluate(x=ann_x_test, y=ann_y_test, verbose=False)
    ann_all_data_alpha_delta_3t1.append(score)


In [23]:
save_ann_data_to(ann_all_data_alpha_delta_3t1, 'ANN_3t1_alpha_delta.xlsx')

In [ ]:
ann_all_data_alpha_delta_25t1 = []
for i in range(100):
    ann_data, labels = cutting(seiz_ad.copy(), non_seiz_ad.copy(), factor=2.5)
    ann_x_train, ann_x_test, ann_y_train, ann_y_test = sklearn.model_selection.train_test_split(ann_data, labels)

    inp = Input(shape=(6,))
    hidden = Dense(6, activation='sigmoid')(inp)
    outp = Dense(1)(hidden)
    model = Model(inputs=inp, outputs=outp)

    model.compile(optimizer='SGD',
                  loss='mean_squared_error',
                  metrics=metrics)  #Using MSE as lossfunction

    model.fit(x=ann_x_train, y=ann_y_train,
              batch_size=3,
              epochs=1000,
              shuffle=True,
              verbose=False)
    score = model.evaluate(x=ann_x_test, y=ann_y_test, verbose=False)
    ann_all_data_alpha_delta_25t1.append(score)


In [25]:
save_ann_data_to(ann_all_data_alpha_delta_25t1, 'ANN_25t1_alpha_delta.xlsx')

In [ ]:
ann_all_data_alpha_delta_2t1 = []
for i in range(100):
    ann_data, labels = cutting(seiz_ad.copy(), non_seiz_ad.copy(), factor=2)
    ann_x_train, ann_x_test, ann_y_train, ann_y_test = sklearn.model_selection.train_test_split(ann_data, labels)

    inp = Input(shape=(6,))
    hidden = Dense(6, activation='sigmoid')(inp)
    outp = Dense(1)(hidden)
    model = Model(inputs=inp, outputs=outp)

    model.compile(optimizer='SGD',
                  loss='mean_squared_error',
                  metrics=metrics)  #Using MSE as lossfunction

    model.fit(x=ann_x_train, y=ann_y_train,
              batch_size=3,
              epochs=1000,
              shuffle=True,
              verbose=False)
    score = model.evaluate(x=ann_x_test, y=ann_y_test, verbose=False)
    ann_all_data_alpha_delta_2t1.append(score)


In [27]:
save_ann_data_to(ann_all_data_alpha_delta_2t1, 'ANN_2t1_alpha_delta.xlsx')

In [ ]:
all_rdf_scores = []
for i in range(100):
    ann_data, labels = norm(seiz_ad.copy(), non_seiz_ad.copy())
    ann_x_train, ann_x_test, ann_y_train, ann_y_test = sklearn.model_selection.train_test_split(ann_data, labels)
    rdf = sklearn.ensemble.RandomForestClassifier()
    rdf.fit(ann_x_train, ann_y_train)
    y_pred = rdf.predict(ann_x_test)

    y_pred = rdf.predict(ann_x_test)
    print(y_pred)
    rdf_cm = confusion_matrix(ann_y_test, y_pred)

    tn, fp, fn, tp = rdf_cm.ravel()

    prec = sklearn.metrics.precision_score(ann_y_test, y_pred)
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)

    all_rdf_scores.append([rdf.score(ann_x_test, ann_y_test), prec, sensitivity, specificity])

save_rdf_data_to(all_rdf_scores,'RDF_alpha_delta_all.xlsx')

In [ ]:
all_rdf_scores_3t1 = []
for i in range(100):
    ann_data, labels = cutting(seiz_ad.copy(), non_seiz_ad.copy(),3)
    ann_x_train, ann_x_test, ann_y_train, ann_y_test = sklearn.model_selection.train_test_split(ann_data, labels)
    rdf = sklearn.ensemble.RandomForestClassifier()
    rdf.fit(ann_x_train, ann_y_train)
    y_pred = rdf.predict(ann_x_test)

    y_pred = rdf.predict(ann_x_test)
    print(y_pred)
    rdf_cm = confusion_matrix(ann_y_test, y_pred)

    tn, fp, fn, tp = rdf_cm.ravel()

    prec = sklearn.metrics.precision_score(ann_y_test, y_pred)
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)

    all_rdf_scores_3t1.append([rdf.score(ann_x_test, ann_y_test), prec, sensitivity, specificity])

save_rdf_data_to(all_rdf_scores_3t1,'RDF_alpha_delta_3t1.xlsx')

In [ ]:
all_rdf_scores_25t1 = []
for i in range(100):
    ann_data, labels = cutting(seiz_ad.copy(), non_seiz_ad.copy(), 2.5)
    ann_x_train, ann_x_test, ann_y_train, ann_y_test = sklearn.model_selection.train_test_split(ann_data, labels)
    rdf = sklearn.ensemble.RandomForestClassifier()
    rdf.fit(ann_x_train, ann_y_train)
    y_pred = rdf.predict(ann_x_test)

    y_pred = rdf.predict(ann_x_test)
    print(y_pred)
    rdf_cm = confusion_matrix(ann_y_test, y_pred)

    tn, fp, fn, tp = rdf_cm.ravel()

    prec = sklearn.metrics.precision_score(ann_y_test, y_pred)
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)

    all_rdf_scores_25t1.append([rdf.score(ann_x_test, ann_y_test), prec, sensitivity, specificity])

save_rdf_data_to(all_rdf_scores_25t1,'RDF_alpha_delta_25t1.xlsx')

In [ ]:
all_rdf_scores_2t1 = []
for i in range(100):
    ann_data, labels = cutting(seiz_ad.copy(), non_seiz_ad.copy(), 2)
    ann_x_train, ann_x_test, ann_y_train, ann_y_test = sklearn.model_selection.train_test_split(ann_data, labels)
    rdf = sklearn.ensemble.RandomForestClassifier()
    rdf.fit(ann_x_train, ann_y_train)
    y_pred = rdf.predict(ann_x_test)

    y_pred = rdf.predict(ann_x_test)
    print(y_pred)
    rdf_cm = confusion_matrix(ann_y_test, y_pred)

    tn, fp, fn, tp = rdf_cm.ravel()

    prec = sklearn.metrics.precision_score(ann_y_test, y_pred)
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)

    all_rdf_scores_2t1.append([rdf.score(ann_x_test, ann_y_test), prec, sensitivity, specificity])

save_rdf_data_to(all_rdf_scores_2t1,'RDF_alpha_delta_2t1.xlsx')

In [ ]:
all_svm_scores = []

for i in range(100):
    ann_data, labels = norm(seiz_ad.copy(), non_seiz_ad.copy())
    ann_x_train, ann_x_test, ann_y_train, ann_y_test = sklearn.model_selection.train_test_split(ann_data, labels)
    
    svm = sklearn.svm.SVC(kernel='rbf')
    svm.fit(ann_x_train,ann_y_train)
    
    y_pred = svm.predict(ann_x_test)
    cm = confusion_matrix(ann_y_test, y_pred)
    tn, fp, fn, tp = cm.ravel()

    sensitivity = tp/(tp+fn)
    specificity = tn/(tn+fp)
    
    svm_y_precision = svm.decision_function(ann_x_test)
    average_svm_precision = sklearn.metrics.average_precision_score(ann_y_test, svm_y_precision)    
    all_svm_scores.append([svm.score(ann_x_test,ann_y_test), average_svm_precision, sensitivity, specificity])
    
save_svm_data_to(all_svm_scores,'SVM_alpha_delta_all.xlsx')

In [ ]:
all_svm_scores_3t1 = []

for i in range(100):
    ann_data, labels = cutting(seiz_ad.copy(), non_seiz_ad.copy(), 3)
    ann_x_train, ann_x_test, ann_y_train, ann_y_test = sklearn.model_selection.train_test_split(ann_data, labels)
    
    svm = sklearn.svm.SVC(kernel='rbf')
    svm.fit(ann_x_train,ann_y_train)
    
    y_pred = svm.predict(ann_x_test)
    cm = confusion_matrix(ann_y_test, y_pred)
    tn, fp, fn, tp = cm.ravel()

    sensitivity = tp/(tp+fn)
    specificity = tn/(tn+fp)
    
    svm_y_precision = svm.decision_function(ann_x_test)
    average_svm_precision = sklearn.metrics.average_precision_score(ann_y_test, svm_y_precision)    
    all_svm_scores_3t1.append([svm.score(ann_x_test,ann_y_test), average_svm_precision, sensitivity, specificity])
    
save_svm_data_to(all_svm_scores_3t1,'SVM_alpha_delta_3t1.xlsx')

In [ ]:
all_svm_scores_25t1 = []

for i in range(100):
    ann_data, labels = cutting(seiz_ad.copy(), non_seiz_ad.copy(), 2.5)
    ann_x_train, ann_x_test, ann_y_train, ann_y_test = sklearn.model_selection.train_test_split(ann_data, labels)
    
    svm = sklearn.svm.SVC(kernel='rbf')
    svm.fit(ann_x_train,ann_y_train)
    
    y_pred = svm.predict(ann_x_test)
    cm = confusion_matrix(ann_y_test, y_pred)
    tn, fp, fn, tp = cm.ravel()

    sensitivity = tp/(tp+fn)
    specificity = tn/(tn+fp)
    
    svm_y_precision = svm.decision_function(ann_x_test)
    average_svm_precision = sklearn.metrics.average_precision_score(ann_y_test, svm_y_precision)    
    all_svm_scores_25t1.append([svm.score(ann_x_test,ann_y_test), average_svm_precision, sensitivity, specificity])
    
save_svm_data_to(all_svm_scores_25t1,'SVM_alpha_delta_25t1.xlsx')

In [ ]:
all_svm_scores_2t1 = []

for i in range(100):
    ann_data, labels = cutting(seiz_ad.copy(), non_seiz_ad.copy(), 2)
    ann_x_train, ann_x_test, ann_y_train, ann_y_test = sklearn.model_selection.train_test_split(ann_data, labels)
    
    svm = sklearn.svm.SVC(kernel='rbf')
    svm.fit(ann_x_train,ann_y_train)
    
    y_pred = svm.predict(ann_x_test)
    cm = confusion_matrix(ann_y_test, y_pred)
    tn, fp, fn, tp = cm.ravel()

    sensitivity = tp/(tp+fn)
    specificity = tn/(tn+fp)
    
    svm_y_precision = svm.decision_function(ann_x_test)
    average_svm_precision = sklearn.metrics.average_precision_score(ann_y_test, svm_y_precision)    
    all_svm_scores_2t1.append([svm.score(ann_x_test,ann_y_test), average_svm_precision, sensitivity, specificity])
    
save_svm_data_to(all_svm_scores_2t1,'SVM_alpha_delta_2t1.xlsx')